In [ ]:
#Detector
!git clone --single-branch --branch  archive https://github.com/ultralytics/yolov3.git

#Download video
!sudo wget https://yt-dl.org/downloads/latest/youtube-dl
!sudo chmod a+rx youtube-dl
!./youtube-dl -f22 -o video.mp4 https://www.youtube.com/watch?v=2RQrKP9a0XE

!pip install GPUtil

Cloning into 'yolov3'...
remote: Enumerating objects: 9466, done.
remote: Total 9466 (delta 0), reused 0 (delta 0), pack-reused 9466
Receiving objects: 100% (9466/9466), 7.83 MiB | 32.48 MiB/s, done.
Resolving deltas: 100% (6483/6483), done.
--2021-03-21 08:47:49--  https://yt-dl.org/downloads/latest/youtube-dl
Resolving yt-dl.org (yt-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yt-dl.org/downloads/2021.03.14/youtube-dl [following]
--2021-03-21 08:47:50--  https://yt-dl.org/downloads/2021.03.14/youtube-dl
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ytdl-org/youtube-dl/releases/download/2021.03.14/youtube-dl [following]
--2021-03-21 08:47:51--  https://github.com/ytdl-org/youtube-dl/releases/download/2021.03.14/youtube-dl
Resolving gith

In [ ]:
import humanize
import GPUtil as GPU
import psutil
import os
import torch

def time_synchronized():
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    return time.time()

def gpu_usage():
    GPUs = GPU.getGPUs()
    # XXX: only one GPU on Colab and isn’t guaranteed
    if len(GPUs) == 0:
        return False
    gpu = GPUs[0]
    process = psutil.Process(os.getpid())
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

!nvidia-smi

Sun Mar 21 08:48:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Init model
import sys
sys.path.insert(0,'/content/yolov3')

from models import *
from utils.utils import *

device = torch.device("cuda:0")

# Default
input_size = (416, 416)
#input_size = (256, 416)  # S4

model = Darknet("yolov3/cfg/yolov3.cfg", input_size ).to(device)

# Load weights
weights = 'yolov3/weights/yolov3.pt'
attempt_download(weights)
model.load_state_dict(torch.load(weights, map_location=device)['model'])

# Half presicion
#model.half() #S2
model.eval()


Model Summary: 222 layers, 6.19491e+07 parameters, 6.19491e+07 gradients


Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (Conv2d): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (Conv2d): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


##Dataloader

How to read live stream?

https://gist.github.com/allskyee/7749b9318e914ca45eb0a1000a81bf56

In [ ]:
import queue
import cv2
import torch
from torch.utils.data import DataLoader
from torch.utils.data import IterableDataset, Dataset
from torchvision import models, datasets, transforms
from PIL import Image
import csv
import numpy as np

class VideoDataset(IterableDataset):
    def __init__(self, filename, input_size):
        super().__init__()
        self.stream = cv2.VideoCapture(filename)
        fps = int(self.stream.get(cv2.CAP_PROP_FPS))
        width  = int(self.stream.get(cv2.CAP_PROP_FRAME_WIDTH))   
        height = int(self.stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print ("FPS {} size {}x{}".format(fps,width,height))

    def read_next_frame(self):
        ret, frame = self.stream.read()
        ret = True
        while ret:
          img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          img = cv2.resize(img,input_size)
          img = transforms.functional.to_tensor(img) #S6
          yield img
          ret, frame = self.stream.read()
        self.stream.release()
        return None

    def __iter__(self):
        return self.read_next_frame()

class FakeDataset(Dataset):
  def __init__(self, input_size):
        super().__init__()
        #self.dry = torch.randn((input_size[1],input_size[0],3)) # For later permute
        self.dry = torch.randn((3,input_size[1],input_size[0]))

  def __getitem__(self,i):
        return self.dry

  def __len__(self):
    return 100000

In [ ]:
import cv2
from torch.utils.data import DataLoader
from torchvision import transforms
import time
import torchvision.transforms.functional as TF

dataset = VideoDataset("video.mp4",input_size)
batch_size = 1
#batch_size = 64 #S1
#batch_size = 128 #S3
dataloader = DataLoader(dataset,  batch_size=batch_size)

MAX_FRAMES = 300
frames_read = 0
tm = time_synchronized()
cnn_tm = 0
nms_tm = 0
base_log = max(50,batch_size)
log_interval = base_log - (base_log % batch_size)

!nvidia-smi

#S5 Dry
#dataloader = DataLoader(FakeDataset(input_size), batch_size=batch_size )

for images in dataloader:
  frames_read += images.shape[0]
  with torch.no_grad():
    images.requires_grad = False
    images = images.to(device)
    #S6 Move conversion to GPU
    #images = images.type(torch.cuda.FloatTensor)
    #images = images/255 
    #images = images.permute(0, 3, 1, 2)

    # Half
    #images = images.half() #S2
    
    t1 = time_synchronized()
    output = model(images)
    detections = output[0]
    # Half
    #detections = output[0].float() #S2
    t2 = time_synchronized()

    cnn_tm += t2-t1
    results = non_max_suppression(detections)

    t3 =time_synchronized()
    nms_tm += t3 - t2
  
  if not frames_read % log_interval :
    print (gpu_usage())
    print ("Processed {} frames. TOTAL_FPS: {:.1f}, YOLO_FPS: {:.1f}, NMS_FPS: {:.1f}" .format(frames_read,log_interval/(time_synchronized()-tm),log_interval/cnn_tm,log_interval/nms_tm))
    tm = time_synchronized()
    cnn_tm = 0
    nms_tm = 0
  if frames_read > MAX_FRAMES:
    break



FPS 29 size 1280x720
Sun Mar 21 09:33:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    31W /  70W |  13638MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                   